# Multilabel Logistic Regression

In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from metrics.Multilabel_classification_metrics import Precision, Accuracy, Recall, F1Measure, Hamming_Loss

import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score

## Load pickles objects

In [6]:
# Dataframe
path_df = "../Pickles/multilabel_binarizer.pickle"
with open(path_df, 'rb') as data:
    multilabel_binarizer = pickle.load(data)

# features_train
path_features_train = "../Pickles/features_train.pickle"
with open(path_features_train, 'rb') as data:
    features_train = pickle.load(data)

# labels_train
path_labels_train = "../Pickles/labels_train.pickle"
with open(path_labels_train, 'rb') as data:
    labels_train = pickle.load(data)

# features_test
path_features_test = "../Pickles/features_test.pickle"
with open(path_features_test, 'rb') as data:
    features_test = pickle.load(data)

# labels_test
path_labels_test = "../Pickles/labels_test.pickle"
with open(path_labels_test, 'rb') as data:
    labels_test = pickle.load(data)

c:\Users\GeorgiVankov\Desktop\AI\Data Mining\BookClassification\venv\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [8]:
clf.fit(features_train, labels_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [9]:
y_pred = clf.predict(features_test)

In [10]:
print(y_pred[95])
print(labels_test[95])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]


In [11]:
multilabel_binarizer.inverse_transform(y_pred)[95]

('Non-fiction',)

In [12]:
F1Measure(labels_test, y_pred)

0.3104961559903005

In [13]:
# predict probabilities
y_pred_prob = clf.predict_proba(features_test)

In [14]:
y_pred_prob

array([[0.01017825, 0.01793589, 0.00815324, ..., 0.01297529, 0.01804664,
        0.04415776],
       [0.01769488, 0.07792384, 0.00764847, ..., 0.04392288, 0.06058901,
        0.05600263],
       [0.02650032, 0.00988674, 0.01164094, ..., 0.0910331 , 0.11533457,
        0.30877677],
       ...,
       [0.0360147 , 0.00885977, 0.00801898, ..., 0.03046394, 0.01978787,
        0.27505813],
       [0.05635281, 0.01437091, 0.0042974 , ..., 0.01225468, 0.01350915,
        0.01726619],
       [0.05028899, 0.02696781, 0.00511875, ..., 0.00840662, 0.01050152,
        0.10853951]])

In [15]:
t = 0.2485 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

In [16]:
# evaluate performance
F1Measure(labels_test, y_pred_new)

0.4998761878832729

In [17]:
# Exact Match Ratio:
accuracy_score(labels_test, y_pred_new)

0.07841001905799075

In [18]:
Accuracy(labels_test, y_pred_new)

0.3922889684052229

In [19]:
Precision(labels_test, y_pred_new)

0.6213569419055397

In [20]:
Recall(labels_test, y_pred_new)

0.4803637872246638